# Projet 1, TP1 - Introduction à la régression linéaire

Au cœur du deep learning sont des **fonctions** :

- image d'un chiffre ➡ le chiffre en question (MNIST)
- labels & bruit blanc ➡ image (modèles de diffusion, e.g. Midjourney)
- fragment de texte ➡ texte complété (auto-régression, e.g. ChatGPT)
- texte dans une langue ➡ texte dans une autre langue (transformers, e.g. [DeepL](https://www.deepl.com/en/blog/how-does-deepl-work))

Le principe est qu'on ne connaît pas *a priori* cette fonction : on la cherche avec, par exemple, un réseau de neurones qui doit «apprendre».
Cette procédure d'apprentissage est [caractérisé par trois choix](https://ml-course.github.io/master/notebooks/01%20-%20Introduction.html#learning-representation-evaluation-optimization) :

1. **Représentation :** Un modèle qui peut être implémenté (l'architecture, le nombre de paramètres...) ;
2. **Évaluation :** Une *loss*, une fonction objectif ou de score, pour évaluer les paramètres ;
3. **Optimisation :** Une manière *efficace* d'ajuster les paramètres.

Dans ce cours, on s'intéresse à **l'apprentissage supervisé**, c'est-à-dire qu'on connait les images de la fonction (les *outputs*) pour certains arguments (certains *inputs*). Ainsi, la partie d'évaluation consistera à vérifier que les *outputs* correspondent dans notre jeu de données.
Pour la représentation, on va commencer par un simple modèle de régression linéaire. On étudiera ensuite deux types de réseaux de neurones : les [perceptrons multi-couches](https://fr.wikipedia.org/wiki/Perceptron_multicouche) et les [transformeurs](https://fr.wikipedia.org/wiki/Transformeur).

Ce premier projet, séparé en 3 TPs, se concentre sur **l'optimisation**. C'est parce que cet élément est complexe qu'on considère un exemple simple à 2 paramètres interprétables, de sorte à mettre la représentation et l'évaluation de côté pour le moment. On va d'abord rapidement introduire le problème pour se motiver, puis introduire des méthodes d'optimisation avec un seul paramètre et enfin introduire un ingrédient majeur du deep learning : la descente de gradient stochastique. Cela permettra aussi de nous acclimater au pipeline PyTorch.

In [3]:
# Quelques imports utiles
import numpy as np
from numpy.polynomial.polynomial import Polynomial

import plotly.graph_objects as go

rng_seed = sum(ord(c) ** 2 for c in "R5.A.12-ModMath")

## Partie 1 - Introduction du problème de régression linéaire

### 1.1. Génération du jeu de données

Cette fonction génère un jeu de données (ou *dataset*), qui consiste en des couples *input, output*. 
Ici, la fonction sous-jacente est probabiliste,
$$ y^{(k)} = f(x^{(k)}) = w x^{(k)} + b + \varepsilon_k , $$
où tous les $\varepsilon_k$ sont tirés aléatoirement et indépendemment, de même loi normale $\varepsilon_k \sim \mathcal{N}(0, \sigma^2)$. 
On n'a pas accès aux $\varepsilon_k$, seulement aux couples $(x^{(k)}, y^{(k)})$. 
Les paramètres $w$ et $b$ s'appellent respectivement la pente (*slope*) et le biais (*bias*).

Par exemple, les données peuvent représenter la taille $x$ et le poids $y$ de différents individus. Il y a des variations intrinsèques que l'on modélise par de l'aléatoire : deux individus de même taille peuvent avoir un poids différent. On regarde ici un problème générique, donc les données ici sont purement fictives, ne représentent rien de particulier, et sont générées automatiquement.

In [23]:
def get_dataset(
    params=(5.0, -2.0),
    x_span=(-3.0, 3.0),
    n_data=100,
    noise=6.0,
    rng=np.random.default_rng(rng_seed),
):
    slope, bias = params
    x = rng.uniform(*x_span, n_data)
    y = slope * x + bias + noise * rng.normal(0.0, 1.0, x.shape)
    return x, y

### 1.2. Affichage du dataset

Supposons maintenant que le jeu de données nous arrive entre les mains. 
En affichant le dataset, on s'informe sur le modèle que l'on souhaite implémenter.
En l'occurrence, on fait un choix de **représentation** assez restrictif, à savoir une dépendance affine :
$$ y \approx f(x; w, b) = w x + b. $$
On pourrait aussi choisir un modèle sous-jacent probabiliste, mais on cherche juste la «meilleure» droite pour le moment, i.e. les «meilleurs» coefficients $(w,b)$.
Il s'agit d'un problème de **régression linéaire**.

*Remarque fondamentale :* La relation entre $x$ et $y$ n’est pas linéaire, mais affine. L'appellation «régression linéaire» provient en fait de la dépendance linéaire de $y$ par rapport aux paramètres $(w, b)$. Contrairement aux apparences, les modèles suivants sont bien des modèles linéaires :
1. $y^{(k)} = w g(x^{(k)}) + b + \varepsilon_k$ ;
2. Modèle de Cobb-Douglas: $y^{(k)} = b \bigl( x^{(k)} \bigr)^w \varepsilon_k$, car $\ln(y^{(k)}) = w \ln(x^{(k)}) + \tilde{b} + \ln(\varepsilon_k)$ avec $\tilde{b} = \ln(b)$ ;
3. Modèle logistique : $y^{(k)} = \frac{\exp( w x^{(k)} + b + \varepsilon_k)}{1 + \exp( w x^{(k)} + b + \varepsilon_k)}$, car $\ln\left(\frac{y^{(k)}}{1 − y^{(k)}}\right) = w x^{(k)} + b + \varepsilon_k$.

Avec la structure [`Polynomial`](https://numpy.org/doc/stable/reference/routines.polynomials.html) de NumPy, calculer la droite de régression linéaire pour le jeu de données généré. Une droite affine est un polynôme de degré 1. Afficher les coefficients de cette droite et la tracer.

In [ ]:
x_data, y_data = get_dataset()

fig = go.Figure()
fig.add_scatter(x=x_data, y=y_data, mode="markers", name="données")

#TODO utiliser Polynomial.fit pour calculer une régression, et l'afficher
affine_fit = ...

# affichage des coefficients
slope_data, bias_data = affine_fit.deriv()(0), affine_fit(0)
print(f"y = {slope_data} * x + {bias_data}")

fig.update_layout(
    width=700,
    height=400,
    margin=dict(l=20, r=20, b=20, t=20),
    xaxis_title="x",
    yaxis_title="y",
)
fig.show()

### 1.3. Définition du problème inverse

Comment déterminer la pente et le biais ? 
Vous avez déjà fait des régressions linéaires, peut-être avec `polyfit`, peut-être avec d'autres méthodes.
Toutes ces méthodes sont basées sur un problème d'optimisation, appelé la **méthode des moindres carrés**, où on cherche $w, b$ qui minimisent la fonctionnelle 

$$ {\rm MSE}(w, b) := \frac{1}{N} \sum_{k = 1}^N \bigl( w x^{(k)} + b - y^{(k)} \bigr)^2 $$

L'acronyme ${\rm MSE}$ signifie *mean squared error*. Affichons cette fonction en fonction des paramètres $(w, b)$.

In [ ]:
# valeurs de pente et de biais 
w = np.linspace(-2, 10, 50)
b = np.linspace(-12, 12, 50)

#TODO calcul de la MSE sous forme de matrice, on a w en colonne et b en ligne
loss = ...

# affichage du graphe
fig = go.Figure()
fig.add_surface(x=w, y=b, z=loss, showscale=False)
fig.update_layout(
    autosize=False,
    width=400,
    height=380,
    margin=dict(l=20, r=20, b=20, t=20),
    scene=dict(xaxis_title="pente", yaxis_title="biais", zaxis_title="erreur"),
)
fig.show()

## Partie 2 - Introduction aux tenseurs

Dans la cellule au-dessus, `w`, `b`, et `loss` sont des **tenseurs**. 
Dans ce contexte informatique, ce terme technique indique simplement des tableaux de nombres structurés, une sorte de généralisation des matrices et des vecteurs. 
Ainsi, `w` et `b` sont des tenseurs d'ordre 1 (i.e. des vecteurs) tandi que `loss` est un tensor d'ordre 2 (i.e. une matrice).
Parfois, on remplace le terme «ordre» par «rang» ou «dimension».

Avec NumPy, la structure s'appelle [`ndarray`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) et avec PyTorch, il s'agit de [`Tensor`](https://pytorch.org/docs/stable/tensors.html).

*Remarque :* Dans la communauté physique & mathématique, le concept de tenseur est différent, mais cela ne nous concerne pas ici.

### 2.1. Le caractère structuré des données

Dans la cellule suivante, quelles sont les listes de listes qui peuvent être converties en tenseurs ? Vérifier votre raisonnement grâce à la fonction de conversion `array` de NumPy. Quel est l'ordre de ces tenseurs ?

In [ ]:
list1 = [[1,2], [3,4,5]]
list2 = [[1,2,3], [4,5,6]]

#TODO conversion en tenseur si possible

### 2.2. Les opérations *componentwise*

La différence majeure avec les matrices et les vecteurs est la structure algébrique : par défaut, les additions, produits, etc sont appliqués terme à terme.
De même pour les fonctions, comme indiqué ci-dessous.

In [ ]:
x = np.linspace(-1, 1, 500)

y1 = np.sin(x)

y2 = np.empty_like(x) # initie un tenseur avec les mêmes dimensions que x
#TODO remplir le vecteur y2 avec une boucle et comparer les résultats

### 2.3. Le *broadcasting*

En utilisant la syntaxe `v = u[:, None]` on converti un tenseur de taille $N_u$ en un tenseur de taille $N_u \times 1$. 

Grâce à cela, on peut calculer `loss` sans boucle dans l'exemple de code ci-dessous. 
Attention, cette version a un coût mémoire *beaucoup* plus élevé ($N_w \times N_b \times N_x$ vs $N_w \times N_b + N_x$).
Cela évite de faire des boucles, qui sont notoirement longues en Python, mais est parfois quand même plus long du fait de ce surcoût mémoire.

**Remarque :** Pour éviter ce surcoût et éviter d'écrire une boucle, on pourrait par exemple utiliser la structure `LazyTensor` du module [`KeOps`](https://www.kernel-operations.io/keops/index.html), mais cela dépasse largement le cadre du cours.

Avec la méthode `shape` (e.g. `u.shape`), étudier la taille de tous les tenseurs impliqués dans ce calcul : `w[None, :, None]`, `x[None, None, :]`, leur produit, `b[:, None, None]`, sa somme avec les autres, etc. 
Les changements de taille sont dûs à des règles de [*broadcasting*](https://numpy.org/doc/stable/user/basics.broadcasting.html) qui deviendront naturelles à force de manipuler ces objets. 
Le plus important pour éviter les erreurs est de s'assurer que **tous les tenseurs sont du même ordre**.

In [ ]:
err = w[None, :, None] * x[None, None, :] + b[:, None, None] - y[None, None, :]
# on prend la moyenne seulement par rapport à la dimension 2, 
# i.e. loss[i,j] est la moyenne par rapport à k des err[i,j,k]
loss = np.mean(err**2, axis=2)

### 2.4. Des questions ?

Pour avoir plus d'exemples, vous pouvez regarder cette [vidéo de deepmath](https://www.youtube.com/watch?v=XoB6xPMZ8y8). Sinon, passez au TP suivant !